# Koopman RL Update (June 10th)

The most recent updates involve testing of the efficacy of the multi-Koopman approach where we have one Koopman operator for each action in an action space. We hoped to see that our operators could approximate the dynamics of CartPole very well. It appears that this is, in fact, the case, and so we should be able to move forward with this.

## Imports:

In [ ]:
import gym
import estimate_L
import numpy as np
import matplotlib.pyplot as plt
from sklearn.kernel_approximation import RBFSampler

## Definitions and Data:

In [ ]:
def l2_norm(true_state, predicted_state):
    return np.sum(np.power(( true_state - predicted_state ), 2 ))

X = np.load('random-agent/cartpole-states.npy').T
Y = np.append(np.roll(X, -1, axis=1)[:,:-1], np.zeros((X.shape[0],1)), axis=1)
U = np.load('random-agent/cartpole-actions.npy').reshape(1,-1)

X_0 = np.load('random-agent/cartpole-states-0.npy').T
Y_0 = np.load('random-agent/cartpole-next-states-0.npy').T
X_1 = np.load('random-agent/cartpole-states-1.npy').T
Y_1 = np.load('random-agent/cartpole-next-states-1.npy').T

state_dim = X.shape[0]

percent_training = 0.8
train_ind = int(np.around(X.shape[1]*percent_training))
X_train = X[:,:train_ind]
Y_train = Y[:,:train_ind]
train_inds = [
    int(np.around(X_0.shape[1]*percent_training)),
    int(np.around(X_1.shape[1]*percent_training))
]
X_0_train = X_0[:,:train_inds[0]]
X_1_train = X_1[:,:train_inds[1]]
Y_0_train = Y_0[:,:train_inds[0]]
Y_1_train = Y_1[:,:train_inds[1]]

The kernel approximator used for this system was the RBF Sampler from scikit learn which uses random fourier features to find a good radial basis function to use. The radial basis function kernel is defined as $$ K(\mathbf{x},\mathbf{x}') = \text{exp} \Bigg(-\frac{||\mathbf{x} - \mathbf{x}'||^2}{2 \sigma^2}\Bigg) $$ Where $\gamma = \frac{1}{2 \sigma^2}$. Since it requires a $\gamma$ value, we have to calculate some good $\gamma$ so that we are not just guessing. http://alex.smola.org/teaching/kernelcourse/day_2.pdf contains a slide on how to find such a $\gamma$ using a so-called "median trick" that can be used as a good solution.

In [ ]:
# Median trick
num_pairs = 1000
pairwise_distances = []
for _ in range(num_pairs):
    i, j = np.random.choice(np.arange(X.shape[1]), 2)
    x_i = X[:,i]
    x_j = X[:,j]
    pairwise_distances.append(np.linalg.norm(x_i - x_j))
pairwise_distances = np.array(pairwise_distances)
gamma = np.quantile(pairwise_distances, 0.9)

# RBF Sampler
rbf_feature = RBFSampler(gamma=gamma, random_state=1)
X_features = rbf_feature.fit_transform(X)
def psi(x):
    return X_features.T @ x.reshape((state_dim,1))

In [ ]:
# Psi matrices
def getPsiMatrix(psi, X):
    k = psi(X[:,0].reshape(-1,1)).shape[0]
    m = X.shape[1]
    matrix = np.empty((k,m))
    for col in range(m):
        matrix[:, col] = psi(X[:, col])[:, 0]
    return matrix

Psi_X = getPsiMatrix(psi, X_train)
Psi_Y = getPsiMatrix(psi, Y_train)

Psi_X_0 = getPsiMatrix(psi, X_0_train)
Psi_Y_0 = getPsiMatrix(psi, Y_0_train)
Psi_X_1 = getPsiMatrix(psi, X_1_train)
Psi_Y_1 = getPsiMatrix(psi, Y_1_train)

Solve for the multiple Koopman operators using the following minimization problem: $\textbf{min}_{K_u}|| \Psi_Y^\top - \Psi_X^\top K_u^\top ||$

In [ ]:
K_0 = estimate_L.rrr(Psi_X_0.T, Psi_Y_0.T).T
K_1 = estimate_L.rrr(Psi_X_1.T, Psi_Y_1.T).T

Find mapping operator $B$ such that $B^\top \psi(x) \approx x$

In [ ]:
B = estimate_L.SINDy(Psi_X.T, X_train.T)

The first thing that we wanted to check in our approach is how well one-step prediction worked. The results are slightly confusing since it isn't great at predicting one-step ahead. Even though it is not a great approximation, it is not terrible either.

### One-step prediction error:

In [ ]:
title = "One-step prediction error:"
print()

# data_point_index = 1000
horizon = 1000
norms = []
# action_path = U[0, data_point_index:data_point_index+horizon]
action_path = U[0, :horizon]
# starting_point = int(np.around(np.random.rand() * X_train.shape[1]))
starting_point = 1700
true_state = X[:,starting_point]
for h in range(horizon):
    action = action_path[h]
    psi_x = psi(true_state)
    predicted_state = B.T @ K_0 @ psi_x if action == 0 else B.T @ K_1 @ psi_x
    true_state = X[:,starting_point+h+1]

    norm = l2_norm(true_state.reshape(-1,1), predicted_state)
    norms.append(norm)

print("Mean norm:", np.mean(norms))
plt.plot(norms, marker='.', linestyle='')
plt.title(title)
plt.ylabel('L2 Norm')
plt.xlabel('Timestep')
plt.show()

Next, we wanted to test was how far out into the future the multi-Koopman approach could predict before it started to deviate heavily from the ground-truth. It looks like it can predict approximately 800 steps ahead which is certainly sufficient for CartPole!

### Prediction compounding error:

In [ ]:
title = "Prediction compounding error:"
print(title)

env = gym.make('CartPole-v0')
horizon = 1000
num_trials = 1#000
norms = []
for i in range(num_trials):
    action_path = [np.random.choice([0,1]) for i in range(horizon)]
    trial_norms = []
    true_state = env.reset()
    predicted_state = true_state.copy()
    for h in range(horizon):
        action = action_path[h]
        psi_x = psi(predicted_state)
        predicted_state = B.T @ K_0 @ psi_x if action == 0 else B.T @ K_1 @ psi_x
        true_state, ___, __, _ = env.step(action)

        norm = l2_norm(true_state.reshape(-1,1), predicted_state)
        trial_norms.append(norm)
    norms.append(trial_norms)

plt.plot(np.mean(norms, axis=0))
plt.title(title)
plt.ylabel('L2 Norm')
plt.xlabel('Timestep')
plt.show()

Then we wanted to check how well our $B$ matrix actually works when trying to retrieve $x$ from $\psi(x)$. The error is very, very low which is great to see.

### Error for $\psi(x) \to x$:

In [ ]:
title = "Error for psi(x) -> x:"
print(title)

# data_point_index = 1000
horizon = 1000
norms = []
# action_path = U[0, data_point_index:data_point_index+horizon]
action_path = U[0, :horizon]
# starting_point = int(np.around(np.random.rand() * X_train.shape[1]))
starting_point = -1000
true_states = X[:,starting_point:]
for true_state in true_states.T:
    true_state = true_state.reshape(-1,1)
    projected_state = B.T @ psi(true_state)

    norm = l2_norm(true_state, projected_state)
    norms.append(norm)

print("Mean norm:", np.mean(norms))
plt.plot(norms, marker='.', linestyle='')
plt.title(title)
plt.ylabel('L2 Norm')
plt.xlabel('Timestep')
plt.show()

Perhaps the most important metric is the error in predicting the next lifted state $\psi(x)'$ from $\psi(x)$. The error is not too bad based on the graph which is good news.

### Error for $\psi(x) \to \psi(x)'$:

In [ ]:
# Koopman from psi(x) -> psi(x)'
# || Y     - X B               ||
# || Psi_Y_i   - K Psi_X_i     ||
# || Psi_Y_i.T - Psi_X_i.T K.T ||
K_0 = estimate_L.rrr(Psi_X_0.T, Psi_Y_0.T).T
K_1 = estimate_L.rrr(Psi_X_1.T, Psi_Y_1.T).T

horizon = 1000
action_path = U[0, -horizon:]
norms = []
true_states = X[:, -horizon:]
for h in range(horizon):
    action = action_path[h]
    true_state = true_states[:,h].reshape(-1,1)
    predicted_state = K_0 @ psi(true_state) if action == 0 else K_1 @ psi(true_state)

    norm = l2_norm(psi(true_state), predicted_state)
    norms.append(norm)

print("Mean norm:", np.mean(norms))
plt.plot(norms, marker='.', linestyle='')
plt.title("Error for psi(x) -> psi(x)':")
plt.ylabel('L2 Norm')
plt.xlabel('Timestep')
plt.show()

We also wanted to see how well going directly from $\psi(x)$ to $x'$ would work. Perhaps unsurprisingly, it has roughly the same error as going from $\psi(x) \to \psi(x)'$, but is slightly worse and technically, this operator would not be considered a "Koopman" operator.

### Error for $\psi(x) \to x'$:

In [ ]:
title = "Error for psi(x) -> x':"
print(title)

# Operator from psi(x) -> x'
# || Y     - X B           ||
# || Y_i   - K Psi_X_i     ||
# || Y_i.T - Psi_X_i.T K.T ||
K_0 = estimate_L.rrr(Psi_X_0.T, Y_0_train.T).T
K_1 = estimate_L.rrr(Psi_X_1.T, Y_1_train.T).T

horizon = 1000
action_path = U[0, -horizon:]
norms = []
true_states = X[:, -horizon:]
for h in range(horizon):
    action = action_path[h]
    true_state = true_states[:,h].reshape(-1,1)
    predicted_state = K_0 @ psi(true_state) if action == 0 else K_1 @ psi(true_state)

    norm = l2_norm(true_state, predicted_state)
    norms.append(norm)

print("Mean norm:", np.mean(norms))
plt.plot(norms, marker='.', linestyle='')
plt.title("Error for psi(x) -> x':")
plt.ylabel('L2 Norm')
plt.xlabel('Timestep')
plt.show()

In the hopes that the residual error could be much easier to predict, we were interested in computing that error. Predicting future states with the residual would mean predicting from $\psi(x)$ to $\psi(x)' - \psi(x)$, and then adding $\psi(x)$ back. Unfortunately this did not work well, but since previous errors were good enough, we should be good to use the traditional Koopman solution without much issue.

### Residual error:

In [ ]:
title = "Residual error:"
print(title)

# residuals_0 = Psi_Y_0 - Psi_X_0
# residuals_1 = Psi_Y_1 - Psi_X_1
residuals = Psi_Y - Psi_X
# psi(x) -> psi(x') - psi(x)
# K_0 = estimate_L.rrr(Psi_X_0.T, residuals_0.T).T
# K_1 = estimate_L.rrr(Psi_X_1.T, residuals_1.T).T
K = estimate_L.rrr(Psi_X.T, residuals.T).T

horizon = 1000
action_path = U[0, -horizon:]
norms = []
true_states = X_train[:, -horizon:]
true_states_prime = Y_train[:, -horizon:]
for h in range(horizon):
    action = action_path[h]

    true_state = true_states[:,h].reshape(-1,1)
    psi_x = psi(true_state)

    predicted_residual = K @ psi_x
    predicted_psi_x_prime = psi_x + predicted_residual
    predicted_x_prime = B.T @ predicted_psi_x_prime

    true_x_prime = true_states_prime[:,h].reshape(-1,1)

    norm = l2_norm(true_x_prime, predicted_x_prime)
    norms.append(norm)

print("Mean norm:", np.mean(norms))
plt.plot(norms, marker='.', linestyle='')
plt.title(title)
plt.ylabel('L2 Norm')
plt.xlabel('Timestep')
plt.show()